In [1]:
!pip install findspark

In [2]:
!pip install pyspark

In [3]:
# Import findspark and initialize.
import findspark
findspark.init()

In [4]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [7]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
Three = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(Three).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [27]:
from pyspark.sql.functions import to_date, col, concat
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?


Four = """
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
"""

# Show the result
spark.sql(Four).show()


AnalysisException: ignored

In [25]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?


Five = """
SELECT
  YEAR(date_built) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
AND floors = 2
AND sqft_living >= 2000
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
"""

# Show the result
spark.sql(Five).show()


AnalysisException: ignored

In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import time
start_time = time.time()

Six = """
SELECT
  ROUND(AVG(view), 2) AS VIEW_RATING
FROM home_sales
WHERE price >= 350000
"""

result = spark.sql(Six)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+
|VIEW_RATING|
+-----------+
|      32.26|
+-----------+

--- 0.8808994293212891 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
Nine = """
SELECT view AS View_Rating, ROUND(AVG(price), 2) AS Average_Price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
"""



print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0001881122589111328 seconds ---


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.mode("overwrite").partitionBy("date_built").parquet("home_sale")

In [16]:
# 11. Read the formatted parquet data.
homes = spark.read.parquet('home_sale')

In [17]:
# 12. Create a temporary table for the parquet data.
homes.createOrReplaceTempView("home_sale")

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
thirteen = """
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sale
GROUP BY view
HAVING AVERAGE_PRICE >= 350000
"""

result = spark.sql(thirteen)
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  85|   1056336.74|
|  65|    736679.93|
|  53|     755214.8|
|  78|   1080649.37|
|  81|   1053472.79|
|  76|   1058802.78|
|  91|   1137372.73|
|  93|   1026006.06|
|  52|    733780.26|
|  86|   1070444.25|
|  94|    1033536.2|
|  57|     734340.5|
|  54|    798684.82|
|  96|   1017815.92|
|  92|    970402.55|
|  64|    767036.67|
|  61|    746877.59|
|  88|   1031719.35|
|  72|    780914.67|
|  59|     791453.0|
+----+-------------+
only showing top 20 rows

--- 2.5386672019958496 seconds ---


In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False